In [1]:
using Pkg
Pkg.status()

Status `~/git/Julia_ML_training/unit5/Project.toml`
⌃ [cbdf2221] AlgebraOfGraphics v0.10.7
  [336ed68f] CSV v0.10.15
⌃ [13f3f980] CairoMakie v0.13.9
  [a93c6f00] DataFrames v1.7.0
  [31c24e10] Distributions v0.25.120
⌃ [587475ba] Flux v0.16.3
  [38e38edf] GLM v1.9.0
  [60bf3e95] GLPK v1.2.1
  [09f84164] HypothesisTests v0.11.5
  [4076af6c] JuMP v1.26.0
  [23fbe1c1] Latexify v0.16.8
⌃ [b2108857] Lux v1.13.0
  [eb30cadb] MLDatasets v0.7.18
  [add582a8] MLJ v0.20.8
  [f1d291b0] MLUtils v0.4.8
⌅ [ee78f7c6] Makie v0.22.9
  [ff71e718] MixedModels v4.35.2
  [6f286f6a] MultivariateStats v0.10.3
  [636a865e] NearestNeighborModels v0.2.3
⌃ [429524aa] Optim v1.12.0
  [3bd65402] Optimisers v0.4.6
  [92933f4c] ProgressMeter v1.10.4
  [ce6b1742] RDatasets v0.7.7
  [2913bbd2] StatsBase v0.34.5
⌃ [0c5d862f] Symbolics v6.40.0
  [9e3dc215] TimeSeries v0.24.2
  [e88e6eb3] Zygote v0.7.10
Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those with ⌅ are res

In [2]:
using Lux
using Zygote # The primary automatic differentiation engine
using Optimisers # The new standard for optimizers

In [3]:
using MLDatasets
using MLUtils # Provides the DataLoader
using Statistics
using Random
using ProgressMeter
using Flux: onehotbatch, onecold, logitcrossentropy # Import the needed utilities from Flux

In [4]:
ProgressMeter.ijulia_behavior(:append);

In [5]:
const rng = Random.default_rng()
Random.seed!(rng, 0);

In [6]:
# Hyperparameters
learning_rate = 0.001
batch_size = 128
num_epochs = 10;

In [7]:
train_x_raw, train_y_raw = MNIST(split=:train)[:]
test_x_raw,  test_y_raw  = MNIST(split=:test)[:];

In [8]:
# Preprocessing is identical
function preprocess_features(x)
    return Float32.(reshape(x, 28*28, :))
end

train_x = preprocess_features(train_x_raw)
test_x = preprocess_features(test_x_raw)
train_y = onehotbatch(train_y_raw, 0:9)
test_y = onehotbatch(test_y_raw, 0:9);

In [9]:
# Use DataLoader from MLUtils
train_loader = DataLoader((train_x, train_y), batchsize=batch_size, shuffle=true);

In [10]:
# --- Model Definition and Initialization (Key Lux difference) ---

# Define the model structure. It looks similar to Flux's Chain.
# Note the `=>` syntax for Dense layers.
model = Chain(
      Dense(28*28 => 128, relu),  # Input: 784 -> Hidden 1: 128 neurons
      Dense(128 => 64, relu),     # Hidden 1: 128 -> Hidden 2: 64 neurons
      Dense(64 => 10)              # Hidden 2: 64 -> Output: 10 logits
)

Chain(
    layer_1 = Dense(784 => 128, relu),  # 100_480 parameters
    layer_2 = Dense(128 => 64, relu),   # 8_256 parameters
    layer_3 = Dense(64 => 10),          # 650 parameters
)         # Total: 109_386 parameters,
          #        plus 0 states.

In [11]:
# In Lux, we explicitly initialize the parameters (ps) and state (st).
# This is a core concept: the model structure is separate from its learnable parts.
ps, st = Lux.setup(rng, model);

In [12]:
# You can inspect the structure of the parameters:
Lux.display(ps)

(layer_1 = (weight = Float32[-0.10921513 0.056196645 … 0.110608466 0.057007626; -0.011075623 -0.112844504 … -0.052739404 0.046212457; … ; 0.040891483 0.018354304 … 0.087309256 0.091325976; -0.011361991 -0.09112135 … -0.122030705 -0.11557871], bias = Float32[0.0125793945, -0.024275485, 0.03424715, 8.1130434f-5, -0.02638943, -0.034375455, -0.010637262, 0.025727345, 0.016990406, 0.011116858  …  -0.031849086, -0.021829436, 0.0001961163, 0.022090366, -0.025813958, -0.007818311, 0.018396122, -0.012626337, 0.01134666, 0.0288197]), layer_2 = (weight = Float32[0.2037279 0.121231 … 0.2081713 -0.26592368; 0.10215899 0.30275932 … -0.13877812 0.28471065; … ; -0.030344477 0.09217859 … 0.0432775 -0.19036081; -0.1642193 0.20556633 … 0.06377395 -0.094860405], bias = Float32[-0.06744722, -0.010483333, 0.08303247, -0.07705071, 0.077265754, -0.00868912, -0.04639269, 0.035928313, 0.0037759468, 0.027657595  …  0.01394165, -0.025066027, 0.06153464, 0.0838878, 0.07016804, 0.005167151, -0.020277672, -0.0531549

In [13]:
# --- Loss Function and Optimizer Setup ---

# The loss function must take the model, parameters, state, and data as arguments.
# It returns the loss value and the (potentially updated) state.
function loss_function(model, ps, st, x, y)
    y_pred, st_new = model(x, ps, st)
    loss = logitcrossentropy(y_pred, y)
    return loss, st_new
end

loss_function (generic function with 1 method)

In [14]:
# Set up the optimizer using Optimisers.jl
# It takes the optimization rule (Adam) and the parameters to create the optimizer state.
opt_state = Optimisers.setup(Adam(learning_rate), ps)

# --- 5Training Loop (Explicitly written) ---
println("\nStarting training...")

for epoch in 1:num_epochs
    p = Progress(length(train_loader); desc="Epoch $epoch: ")
    
    for (x_batch, y_batch) in train_loader
        # Zygote.withgradient calculates the loss, the new state, and the gradients all at once.
        (loss, st), grads = Zygote.withgradient(ps -> loss_function(model, ps, st, x_batch, y_batch), ps)
        
        # Update the optimizer state and the parameters
        opt_state, ps = Optimisers.update!(opt_state, ps, grads[1])
        
        next!(p; showvalues=[(:loss, loss)]) # Update progress bar
    end

    # Evaluation step after each epoch
    # In evaluation mode, we don't need gradients.
    # The model call still requires `ps` and `st`.
    y_hat_logits, _ = model(test_x, ps, st)
    y_hat_labels = onecold(y_hat_logits) .- 1 # onecold is 1-based, labels are 0-based
    
    current_accuracy = mean(y_hat_labels .== test_y_raw)
    println("\nEpoch $epoch: Test Accuracy = ", round(current_accuracy * 100, digits=2), "%")
end

println("\nTraining complete!")


Starting training...


Epoch 1:   0%|▏                                         |  ETA: 0:27:44
4m   loss: 2.4095874
Epoch 1: 100%|██████████████████████████████████████████| Time: 0:00:07
   loss: 0.13186748



Epoch 1: Test Accuracy = 95.33%


Epoch 2:  19%|███████▉                                  |  ETA: 0:00:00
4m   loss: 0.07697785
Epoch 2:  40%|████████████████▊                         |  ETA: 0:00:00
4m   loss: 0.13621089
Epoch 2:  62%|██████████████████████████                |  ETA: 0:00:00
4m   loss: 0.124958545
Epoch 2:  84%|███████████████████████████████████▎      |  ETA: 0:00:00
4m   loss: 0.0916301
Epoch 2: 100%|██████████████████████████████████████████| Time: 0:00:00
   loss: 0.11521212



Epoch 2: Test Accuracy = 96.64%


...


Epoch 8: Test Accuracy = 97.54%


Epoch 9:  20%|████████▌                                 |  ETA: 0:00:00
4m   loss: 0.014968177
Epoch 9:  41%|█████████████████                         |  ETA: 0:00:00
4m   loss: 0.017752368
Epoch 9:  62%|█████████████████████████▉                |  ETA: 0:00:00
4m   loss: 0.014934659
Epoch 9:  83%|██████████████████████████████████▊       |  ETA: 0:00:00
4m   loss: 0.009883834
Epoch 9: 100%|██████████████████████████████████████████| Time: 0:00:00
   loss: 0.016362146



Epoch 9: Test Accuracy = 97.84%


Epoch 10:  21%|████████▋                                |  ETA: 0:00:00
4m   loss: 0.012205366
Epoch 10:  42%|█████████████████▎                       |  ETA: 0:00:00
4m   loss: 0.0042203614
Epoch 10:  65%|██████████████████████████▋              |  ETA: 0:00:00
4m   loss: 0.048551943
Epoch 10:  88%|███████████████████████████████████▉     |  ETA: 0:00:00
4m   loss: 0.0048665786
Epoch 10: 100%|█████████████████████████████████████████| Time: 0:00:00
   loss: 0.033069585



Epoch 10: Test Accuracy = 97.39%

Training complete!


In [15]:
# --- Final Evaluation ---
println("\nEvaluating final model on the test set...")
final_logits, _ = model(test_x, ps, st)
final_predictions = onecold(final_logits) .- 1
final_accuracy = mean(final_predictions .== test_y_raw)

println("-------------------------------------------")
println("Final Test Accuracy: ", round(final_accuracy * 100, digits=2), "%")
println("-------------------------------------------")


Evaluating final model on the test set...
-------------------------------------------
Final Test Accuracy: 97.39%
-------------------------------------------
